In [1]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [3]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: shuffled_binary_fullrank
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:07<00:00, 1266.33it/s]


{'basic': {'expected_sample_burst_size': 5.496681559883746,
           'expected_sample_data_size': 4.003838979742333,
           'expected_time_to_recover': 42.689299999999996,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.10894254217695841,
             'bit_redundancy': 190.19309567122582,
             'ideal_bit_efficency': 0.2585046164014023,
             'ideal_bit_redundancy': 66.70013171433695,
             'ideal_packet_efficiency': 1.2925230820070117,
             'ideal_packet_redundancy': 13.340026342867388,
             'ideal_time_to_recover': 17.99070720862222,
             'ideal_transmitted_bits': 89.9535360431111,
             'packet_efficiency': 0.5447127108847921,
             'packet_redundancy': 38.03861913424517,
             'permeability': 0.4214336428243663,
             'saturation': 4.650680865754831,
             'transmitted_bits': 213.4465}}
Found implementation: single_binary_fullrank
  Firs

benchmark: 100%|██████████| 10000/10000 [00:02<00:00, 3486.13it/s]


{'basic': {'expected_sample_burst_size': 5.523003720663995,
           'expected_sample_data_size': 4.0010732684602175,
           'expected_time_to_recover': 25.429,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.18288886176235128,
             'bit_redundancy': 103.89159567122584,
             'ideal_bit_efficency': 0.43534508943103295,
             'ideal_bit_redundancy': 30.160323983091736,
             'ideal_packet_efficiency': 2.1767254471551647,
             'ideal_packet_redundancy': 6.0320647966183465,
             'ideal_time_to_recover': 10.682745662373177,
             'ideal_transmitted_bits': 53.41372831186589,
             'packet_efficiency': 0.9144443088117564,
             'packet_redundancy': 20.778319134245166,
             'permeability': 0.42010089513442045,
             'saturation': 4.650680865754831,
             'transmitted_bits': 127.145}}
